In [6]:
from utils import go_to_project_root
from scipy.stats import mode
import data
import os
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, accuracy_score
from sklearn.model_selection import KFold, train_test_split
from matplotlib import pyplot as plt

from sklearn.neural_network import MLPClassifier, BernoulliRBM
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, AdaBoostRegressor, GradientBoostingClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
import sklearn.svm
import keras
from sklearn.feature_selection import RFE
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import xgboost as xgb

In [7]:
def classify(classifier, xtrain, ytrain, xtest, ytest):
    classifier.fit(xtrain, ytrain)
    imp = classifier.feature_importances_
    pred = predict(classifier, xtest)
    return pred

def feature_elim(classifier, xtrain, ytrain):
    rfe = RFE(estimator=classifier, n_features_to_select=n_features, step=10)
    rfe.fit(xtrain, ytrain)
    return rfe.ranking_

def get_new_xtests(path): 
    path = data_root[:-1] + "s/" + path
    xtest = []
    for file in os.listdir(path + "X_test/"):
        xtest += [pd.read_csv(path + "X_test/" + file, index_col=0).to_numpy()[:, [2, 3, 4, 5]]]
    return xtest

def read_data(_path):
    path = data_root + _path
    xtrain = pd.read_csv(path + "X_train.csv", index_col=0).to_numpy()[:200].astype(float)
    ytrain = pd.read_csv(path + "y_train.csv", index_col=0).to_numpy()[:200]
    ytest = pd.read_csv(path + "y_test.csv", index_col=0).to_numpy()
    xtest = []
    for file in os.listdir(path + "X_test/"):
        xtest += [pd.read_csv(path + "X_test/" + file, index_col=0).to_numpy()]

    new_xt = get_new_xtests(_path)
    mean_xt = [np.mean(xt[0], axis=0) for xt in new_xt]

    for i, x in enumerate(xtest):
        xtest[i][:,[2, 3, 4, 5]] = mean_xt[i]

    return xtrain, ytrain, xtest, ytest

def predict(classifier, xtest):
    majority_vote_preds = []
    for x in xtest:
        x = x
        majority_vote_preds += [np.sum(classifier.predict(x).astype(int)) > 1]
    return majority_vote_preds

In [8]:
go_to_project_root()
data_root = "data/processed/800/"
datasets = [read_data(f"K{k+1}/") for k in range(3)]

In [9]:
feature_sets = {
    # "lexical": [1],
    "semantic": [2, 3, 4, 5],
    # "clusters": [6, 7],
    # "nonling":  list(range(8, 30)),
    # "pos": [31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47],
    # "emotion": [47, 48, 49, 50, 51, 52, 53, 54, 55, 56],
    # "embeddings": list(range(57, 357))
}

In [10]:
# ada = []
# fs = feature_sets["embeddings"] + feature_sets["emotion"] + feature_sets["pos"]
_del = feature_sets["semantic"]
f = _del

mean = 0
for i in range(3):
    xtrain, ytrain, xtest, ytest = datasets[i]
    c = GradientBoostingClassifier()
    # pred = classify(c, np.delete(xtrain, _del, axis=1), ytrain, xtest, ytest)
    pred = classify(c, xtrain, ytrain, xtest, ytest)
    acc = balanced_accuracy_score(ytest, pred)
    mean += acc
print(mean / 3)

0.6999999999999998


In [11]:
mismatch = [x for x, p in enumerate([int(x == True) for x in pred]) if p != ytest[x]]

In [12]:
mismatch

[0,
 1,
 9,
 12,
 13,
 14,
 15,
 16,
 22,
 25,
 26,
 30,
 33,
 35,
 36,
 40,
 41,
 42,
 43,
 53,
 58,
 64,
 66,
 69,
 70,
 72,
 80,
 90,
 93,
 94,
 95,
 98]

In [13]:
pd.DataFrame([xtest[m][0] for m in mismatch])

,0,1,2,3,4,5,6,7,8,9,...,347,348,349,350,351,352,353,354,355,356
0,7.50,0.98,0.580955,0.580955,0.580955,0.580955,0.00,0.42,1,1.00,...,0.284624,-0.013412,-0.195229,0.184144,0.188326,-0.020228,0.040004,-0.008719,-0.048525,0.054909
1,8.56,1.00,0.619110,0.619110,0.619110,0.619110,0.02,0.44,2,1.56,...,0.260419,-0.010028,-0.174950,0.140882,0.109642,0.052822,0.011637,-0.034743,-0.029555,0.092662
2,8.82,1.00,0.640548,0.640548,0.640548,0.640548,0.02,0.38,2,1.96,...,0.198711,-0.000020,-0.145692,0.158541,0.117234,0.100025,-0.045165,0.055751,-0.059186,0.031795
3,11.14,1.00,0.648598,0.648598,0.648598,0.648598,0.12,0.32,2,1.06,...,0.209349,0.012015,-0.176687,0.286394,0.081167,0.059793,-0.025541,0.060909,-0.089637,0.010788
4,9.14,0.92,0.598249,0.598249,0.598249,0.598249,0.00,0.52,1,1.00,...,0.281274,-0.022952,-0.132948,0.210196,0.102468,0.023697,0.003421,-0.011731,-0.110694,0.041663
5,9.68,1.00,0.579951,0.579951,0.579951,0.579951,0.08,0.52,1,1.00,...,0.288692,-0.031779,-0.160704,0.212836,0.178110,0.068823,0.014393,-0.012220,-0.123156,0.017638
6,6.68,1.00,0.563152,0.563152,0.563152,0.563152,0.04,0.38,1,1.00,...,0.282537,0.003542,-0.088421,0.117531,0.100682,0.071690,-0.022632,0.019563,-0.076970,0.059577
7,9.66,1.00,0.597910,0.597910,0.597910,0.597910,0.08,0.58,1,1.00,...,0.283849,-0.040608,-0.153684,0.196859,0.214526,0.037496,0.033136,-0.019455,-0.075675,0.037446
8,8.64,1.00,0.526621,0.526621,0.526621,0.526621,0.18,0.48,2,1.02,...,0.227369,0.002175,-0.141336,0.102787,0.127321,0.059000,-0.037056,0.006883,-0.060917,0.081950
9,6.18,0.98,0.630640,0.630640,0.630640,0.630640,0.08,0.34,2,0.92,...,0.283571,0.020827,-0.149753,0.219545,0.085846,0.129766,-0.022316,0.037707,-0.102794,-0.003213


In [14]:
plt.figure(figsize=(12, 8))
plt.plot(range(5, 300, 5), extra, label="Extra Trees")
plt.plot(range(5, 300, 5), logs, label="Logistic Regression")
plt.plot(range(5, 300, 5), randomf, label="Random Forest")
# plt.plot(range(5, 300, 5), xgboost, label="XGBoost")

plt.legend(fontsize=12)
plt.xlabel("Number of Selected Features", fontsize=20)
plt.ylabel("Testing accuracy", fontsize=20)

NameError: name 'extra' is not defined

In [15]:
0.68 + 0.73 + 0.6200000000000001

2.0300000000000002

In [16]:
f = _del
for i in range(3):
    xtrain, ytrain, xtest, ytest = datasets[i]
    mean, _  = classify(c, np.delete(xtrain, f, axis=1), ytrain, xtest, ytest)
    print(mean)

ValueError: Number of features of the model must match the input. Model n_features is 353 and input n_features is 357 

In [17]:
list(feature_sets.values())

[[2, 3, 4, 5]]